# cap c3
- be very careful!!!!

In [1]:
import numpy as np
import zarr
from numcodecs import GZip 
# alignment functions
import tifffile

In [2]:
p = '/u/home/f/f7xiesnm/project-zipursky/easifish/lt186/outputs/'
outdir = '/u/home/f/f7xiesnm/project-zipursky/easifish/results/test_bigstream_n5cap'
!mkdir -p $outdir

In [3]:
def cap_img(a0, pctl=99.9):
    """
    """
    a = np.clip(a0, None, np.percentile(a0, pctl)).astype('uint16')
    print(np.max(a0), '->', np.max(a))
    return a

def modify_n5_like_a0(img_path, a0, a, a_slot):
    """
    be very careful - 
    """
    n5_root = zarr.open_group(store=zarr.N5Store(img_path), mode='a')
    dataset = n5_root.require_dataset(
        a_slot,
        shape=a0.shape,
        chunks=a0._chunks,
        dtype=a0.dtype,
        compressor=GZip(level=-1),
        data=a)
    # set additional attributes
    dataset.attrs.update(**a0.attrs.asdict())
    
    return 

In [12]:
img_path = p + '/r2/stitching/export.n5'
out_cap = outdir+'/r2c3s4_cap.tiff'
out_org = outdir+'/r2c3s4_org.tiff'

In [5]:
!ls $img_path

attributes.json  c0  c1  c2  c3_orig  c4


In [6]:
%%time
g0_slot = 'c3_orig' 
g_slot  = 'c3'

# update dataset - s4 resolution
a0 = zarr.open(store=zarr.N5Store(img_path), mode='r')[g0_slot+'/s4']
a = cap_img(a0[...], pctl=99.9)
modify_n5_like_a0(img_path, a0, a, g_slot+'/s4')

# update dataset - s3 resolution
a0 = zarr.open(store=zarr.N5Store(img_path), mode='r')[g0_slot+'/s3']
a = cap_img(a0[...], pctl=99.9)
modify_n5_like_a0(img_path, a0, a, g_slot+'/s3')

# update group attrs
g0 = zarr.open(store=zarr.N5Store(img_path), mode='r')[g0_slot]
g  = zarr.open(store=zarr.N5Store(img_path), mode='a')[g_slot]
g.attrs.update(g0.attrs.asdict())

47284 -> 239
61437 -> 247
CPU times: user 1min 51s, sys: 4.75 s, total: 1min 56s
Wall time: 2min 3s


# test out

In [10]:
zarr_handle = zarr.open(store=zarr.N5Store(img_path), mode='r')
zarr_handle['c3/s4']

<zarr.core.Array '/c3/s4' (250, 459, 565) uint16 read-only>

In [13]:
tifffile.imwrite(out_cap, zarr_handle['c3/s4'][...])
tifffile.imwrite(out_org, zarr_handle['c3_orig/s4'][...])